In [1]:
 
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

Using cache found in /Users/alexander/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [2]:
import urllib
url, filename = ("https://github.com/mateuszbuda/brain-segmentation-pytorch/raw/master/assets/TCGA_CS_4944.png", "TCGA_CS_4944.png")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [3]:
import numpy as np
from PIL import Image
from torchvision import transforms

input_image = Image.open(filename)
m, s = np.mean(input_image, axis=(0, 1)), np.std(input_image, axis=(0, 1))
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=m, std=s),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model = model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

print(torch.round(output[0]))

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


In [4]:
from task1pack.utils.data import SegmentationDataset

#pip install -e .
#pip install  abbyy_course_cvdl_t2

from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
import wandb

import torch

from task1pack.utils.data import HeatmapDataset

In [5]:
DATASET_PATH = Path() / '..' / '..' / 'midv500_compressed'
#DATASET_PATH = Path() / '..' / '..' / '..' / '..' / '..' / '..' / 'Downloads' / 'midv500_compressed'
assert DATASET_PATH.exists(), DATASET_PATH.absolute()

# Собираем список пакетов (MidvPackage) 
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_ocr_t1.data.MidvPackage)

In [6]:
from torchvision.transforms import Resize, Compose, ToTensor

IMAGE_SIZE = [416, 416]
HEATMAP_SIZE = [104, 104]

image_transforms = Compose([
    ToTensor(),
    Resize(IMAGE_SIZE),
])

target_transforms = Compose([
    Resize(IMAGE_SIZE),
])

train_dataset = SegmentationDataset(data_packs=data_packs, 
        split='train', image_transforms=image_transforms, target_transforms=target_transforms)
test_dataset = SegmentationDataset(data_packs=data_packs, 
        split='test', image_transforms=image_transforms, target_transforms=target_transforms)

print(len(train_dataset), len(test_dataset))
print(train_dataset[0][0].shape, train_dataset[0][1].shape)

10750 4250
torch.Size([3, 416, 416]) torch.Size([1, 416, 416])


/Applications/anaconda3/envs/ml/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [7]:
from task1pack.utils.train import train_model, show_train_plots

In [8]:
from torch.nn import BCELoss

In [9]:
train_dataloader_config = {
    'batch_size': 24,
    'shuffle': True,
}

test_dataloader_config = {
    'batch_size': 128,
    'shuffle': False,
}

training_config = {
    'lr': 1e-3,
    'epochs': 100,
    'step_size': 15,
    'gamma': 0.1,
}

device = 'cpu'
criterion = BCELoss()

In [10]:
train_losses, test_losses, trained_model = train_model(
    train_dataset=train_dataset, 
    test_dataset=test_dataset, 
    model=model, 
    train_dataloader_kwargs=train_dataloader_config, 
    test_dataloader_kwargs=test_dataloader_config, 
    training_kwargs=training_config,
    criterion=criterion,
    device=device,
    wandb_instance=None,
)

KeyboardInterrupt: 